# Todo


- Implement LGS ✅
- Implement VSGS ✅
- Explore multiple dataset generation approach ✅ (Just scenario 3 left, I am not sure if it is important)
- Show results ✅
- Implement PSD estimation algorithm (currently the true PSD is used) ✅
- Compare the "graph baseline" with traditional kernel methods ~ need some work to find penalty parameter

In [ ]:
import numpy as np
import pygsp
import ruptures as rpt
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression
from ruptures.metrics import *
from tqdm import tqdm
from scipy.stats import gamma

SEED = 8976813
np.random.seed(SEED)

In [ ]:
def metrics(true_bkps, bkps, verbose=True):
    p, r = rpt.metrics.precision_recall(true_bkps, bkps)
    hausdorff = rpt.metrics.hausdorff(true_bkps, bkps)
    randindex = rpt.metrics.randindex(true_bkps, bkps)
    hamming = rpt.metrics.hamming(true_bkps, bkps)
    if verbose:
        print(f"""
            Hausdorff: {hausdorff:.2f}, 
            Precision: {p:.2f}, 
            Recall: {r:.2f}, 
            Randindex: {randindex:.2f}, 
            Hamming: {hamming:.2f}
        """)
    return p, r, hausdorff, randindex

# Dataset

In [ ]:
def generate_barabasi_albert(
        n_nodes,
        m_barabasi=4,
        mean_exponential=20,
        random_freq=20
    ):
    # Generate Graph Structure
    G = pygsp.graphs.BarabasiAlbert(N=n_nodes, m0=m_barabasi, m=m_barabasi, seed=SEED)
    G.set_coordinates()
    G.compute_fourier_basis()

    spectral_profile=lambda x: 2*gamma.pdf(x,a=20.,loc=5.)+1.0
    H = pygsp.filters.Filter(G, lambda x: spectral_profile(x))
    PSD = (spectral_profile(G.e))**2

    # Generate Signals
    n_bkps = 4
    bkps = []
    mus = []
    signals = []
    
    acc_bkp = 0
    acc_mean = np.concatenate((np.random.uniform(-5.0,5.0,size=random_freq),np.zeros(n_nodes-random_freq)))
    acc_mean = G.igft(acc_mean).reshape(-1, 1)
    # Make a list of changing points after each breakpoints
    changing_nodes = []
    changing_nodes.append(tuple([[]])) # In the first part, no mean are changed
    max_d = np.argmax(G.d)
    max_d_neighbors = np.where(G.W.todense()[max_d] != 0)[1]
    changing_nodes.append(np.concatenate([max_d_neighbors, [max_d]])) # After the first break, node with maximum degree and neighbord change
    changing_nodes.append(np.argpartition(G.d, -5)[-5:]) # After the second break, 5 nodes with highest degree change
    changing_nodes.append(np.random.choice(range(n_nodes), size=random_freq, replace=False)) # After last break, 20 random nodes are changes

    for i in range(n_bkps):
        # Generate segment
        segment_length = int(np.random.exponential(scale=mean_exponential)+30)
        acc_bkp += segment_length
        acc_mean[changing_nodes[i]] = np.random.uniform(-5.0,5.0,size=(len(changing_nodes[i]),1)) # Changing mean of corresponding nodes
        signal = np.random.normal(size=(n_nodes,segment_length))
        signal = H.filter(signal) + acc_mean

        # Record bkp position, mean and signal
        bkps.append(acc_bkp)
        mus.append(np.copy(acc_mean))
        signals.append(signal)

    signal = np.concatenate(signals, 1)
    return G, PSD, bkps, mus, signal

# Params
n_nodes = 500   
G, PSD, true_bkps, mus, signal = generate_barabasi_albert(n_nodes)
print(f'{len(true_bkps)} changes points')
rpt.display(signal[:5].T, true_bkps)
plt.show()

In [ ]:
def generate_erdos_renyi(
        n_nodes,
        p=0.3,
        mean_exponential=20,
        mean_change_points=3,
        random_freq=20
    ):
    # Generate Graph Structure
    G = pygsp.graphs.ErdosRenyi(N=n_nodes, p=p, seed=SEED)
    G.set_coordinates()
    G.compute_fourier_basis()

    spectral_profile = lambda x: np.sqrt(15)/(np.log(x+10)+1)
    H = pygsp.filters.Filter(G, lambda x: spectral_profile(x))
    PSD = (spectral_profile(G.e))**2

    # Generate Signals
    n_bkps = np.random.poisson(lam=mean_change_points) + 2
    bkps = []
    mus = []
    signals = []
    
    acc_bkp = 0
    acc_mean = np.zeros((n_nodes,1))
    for bkpt in range(n_bkps):
        # Get new segment mean
        subset_freq = np.random.choice(range(n_nodes), size=random_freq, replace=False)
        if bkpt == 0:
            subset_freq = np.arange(random_freq)
        acc_mean[subset_freq] = np.random.uniform(-5.0,5.0,size=(random_freq,1))

        # Generate segment
        segment_length = int(np.random.exponential(scale=mean_exponential)+30)
        acc_bkp += segment_length
        signal = np.random.normal(size=(n_nodes,segment_length)) # Ici on devrait avoir uniform et pas normal
        signal = H.filter(signal) + G.igft(acc_mean)
        # Record bkp position, mean and signal
        bkps.append(acc_bkp)
        mus.append(np.copy(acc_mean))
        signals.append(signal)
    signal = np.concatenate(signals, 1)
    return G, PSD, bkps, mus, signal

# Params
n_nodes = 500
G, PSD, true_bkps, mus, signal = generate_erdos_renyi(n_nodes)
print(f'{len(true_bkps)} changes points')
rpt.display(signal[:5].T, true_bkps)
plt.show()

# PSD Estimation

In [ ]:
def gaussian_kernel(x, m, tau, sig_2):
    return np.exp( -np.square(x - m * tau) / sig_2)

def estimate_PSD_cov(signal, G):
    """This functions allows to estimate the PSD of the graphs signals using covariance estimation.

    Args:
        signal: array, shape (n_nodes, T) the stream of graph signals.
        G: pygsp.graphs,  the graph

    Returns:
        PSD : array, shape (n_nodes) : the PSD of the graphs signals 

    """

    cov = np.cov(signal)
    PSD = np.diagonal(np.conj(G.U).T @ cov @ G.U)
    return PSD


def estimate_PSD_perraudin(signal, G, M=100, filtering_method='chebyshev', exact_window_norm=False, interp_degree=15 ):
    """This functions allows to estimate the PSD of the graphs signals using perraudin method.

    Args:
        signal: array, shape (n_nodes, T) the stream of graph signals.
        G: pygsp.graphs,  the graph
        M : int, number of filters to use
        filtering_method : string, method to use for filtering graphs, either "exact" or "chebyshev"
        window_norm : bool, use exact computation of the window norm
        interp_degree: int, polynomial degree for the interpolation 

    Returns:
        PSD : array, shape (n_nodes) : the PSD of the graphs signals 

    """
    np.random.seed(SEED)
    tau = (M+1) * G.lmax / np.square(M)
    sig_2 = tau
    psd = []
    
    if not exact_window_norm:
        noise = np.random.normal(size=(signal.shape[0], 10))

    for m in range(M):
        filter = lambda x : gaussian_kernel(x, m, tau, sig_2)
        G_filter = pygsp.filters.Filter(G, filter)
        
        if exact_window_norm:
            window_norm = np.sum(np.square(filter(G.e))) + 1e-16
        else:
            filterned_noise = G_filter.filter(noise, method=filtering_method)
            window_norm = np.mean(np.sum(np.square(filterned_noise), axis=0))


        filtered_signal = G_filter.filter(signal, method=filtering_method)
        filtered_signal_norm = np.mean(np.sum(np.square(filtered_signal), axis=0))
        
        psd.append(filtered_signal_norm / window_norm)

    xs = tau * np.arange(M)

    coeff=np.polyfit(xs,np.array(psd),deg=interp_degree)
    p = np.poly1d(coeff)

    PSD=p(G.e)
    return PSD
    

In [ ]:
# Compare True PSD and approximation
w = 30
psd_covariance = estimate_PSD_cov(signal[:,:w], G)
psd_perraudin = estimate_PSD_perraudin(signal[:,:w], G)

fig = plt.figure(figsize=(10, 5)) 
plt.plot(G.e, psd_covariance, label="PSD covariance")
plt.plot(G.e, psd_perraudin, label="PSD peraudin")
plt.plot(G.e, PSD, label="True PSD")
plt.xlabel("lambda")
plt.ylabel("PSD")
plt.legend()
plt.show()

# LGS

In [ ]:
class CostFunctionLGS(rpt.base.BaseCost):
    model = "lgs"
    min_size = 2
    def __init__(self, psd, lbd):
        self.lbd = lbd
        self.signal = None
        self.psd = psd
        
    def fit(self, signal):
        self.signal = signal
        return self
        
    def error(self, start, end):
        y_tilda_bar = np.mean(self.signal[start:end],axis=0)
        temp = np.abs(y_tilda_bar) - 0.5*self.lbd*self.psd
        mu_tilda_bar = np.sign(y_tilda_bar) * np.maximum(temp,0)
        
        sub_sig = np.square(self.signal[start:end] - mu_tilda_bar) / self.psd
        sub_sig += self.lbd * np.abs(mu_tilda_bar)
        
        return np.sum(sub_sig)

def LGS(G, signal, PSD, d_max=75, lbd=1., verbose=True):
    c1, c2 = 500., 100.
    T = signal.shape[1]
    GFT = signal @ G.U
    PSD = PSD
    cost = CostFunctionLGS(PSD, lbd).fit(GFT)
    algo = rpt.Dynp(custom_cost=cost).fit(GFT)
    best_cost = float('inf')
    best_bkps = None
    loop = tqdm(range(1,d_max+1)) if verbose else range(1,d_max+1)
    for d in loop:
        bkps = algo.predict(n_bkps=d)
        curr_cost = cost.sum_of_costs(bkps) + d * (c1 + c2 * np.log(T/d))
        if curr_cost < best_cost:
            best_cost = curr_cost
            best_bkps = bkps
    return best_bkps

In [ ]:
# Results using True PSD
T = signal.shape[1]
d_max = int(T/np.log(T))
bkps = LGS(G, signal.T, PSD, d_max=d_max, lbd=10.)
print(f'{len(bkps)} changes points')
metrics(true_bkps, bkps)
rpt.display(signal[:5].T, true_bkps, bkps)
plt.show()

In [ ]:
# Results using PSD estimation through covariance estimation
T = signal.shape[1]
d_max = int(T/np.log(T))
bkps = LGS(G, signal.T, psd_covariance, d_max=d_max, lbd=10.)
print(f'{len(bkps)} changes points')
metrics(true_bkps, bkps)
rpt.display(signal[:5].T, true_bkps, bkps)
plt.show()

In [ ]:
# Results using PSD estimation with perraudin methods
T = signal.shape[1]
d_max = int(T/np.log(T))
bkps = LGS(G, signal.T, psd_perraudin, d_max=d_max, lbd=10.)
print(f'{len(bkps)} changes points')
metrics(true_bkps, bkps)
rpt.display(signal[:5].T, true_bkps, bkps)
plt.show()

# VSGS

In [ ]:
def lassoVSGS(signal, PSD, lbd):
    """
    Solves the lasso problem from Algorithm2 VSGS
    Solution is computed via a closed form
    """
    emp_mean = np.mean(signal, axis=0)
    temp = np.abs(emp_mean) - 0.5*lbd*PSD
    mu = np.sign(emp_mean) * np.maximum(temp,0)
    return mu

class CostFunctionVSGS(rpt.base.BaseCost):
    model = "vsgs"
    min_size = 2
    def __init__(self, psd, filter_mask):
        self.signal = None
        self.filter_mask = filter_mask
        self.psd = psd
        
    def fit(self, signal):
        self.signal = signal
        return self
        
    def error(self, start, end):
        emp_mean = np.mean(self.signal[start:end],axis=0)
        emp_mean *= self.filter_mask
        temp = np.square(self.signal[start:end] - emp_mean) / self.psd
        return np.sum(temp)

def slopeHeuristic(parameters, T):
    X = np.zeros((len(parameters), 3))
    X[:,0] = parameters[:,2]
    X[:,1] = parameters[:,1]
    X[:,2] = parameters[:,1] * np.log(T / parameters[:,1])
    X /= T
    y = parameters[:,3] / T
    constants = -2 * LinearRegression().fit(X, y).coef_
    return constants.tolist()

def VSGS(G, signal, PSD, lambda_set, d_max=75, verbose=True):
    GFT = signal @ G.U
    PSD = PSD
    T = len(signal)
    parameters_list = np.zeros((len(lambda_set)*d_max, 4)) # Array storing lbd, Dm, d, cost for each iteration
    bkps_list = [] # List storing bkps for each set of parameters
    count = 0
    loop = tqdm(lambda_set) if verbose else lambda_set
    for lbd in loop:
        mu = lassoVSGS(GFT, PSD, lbd)
        filter_freq = np.nonzero(np.abs(mu))[0] # Frequencies kept by Lasso
        filter_mask = (np.abs(mu) > 0) * 1. # Mask = 1 when a frequency is kept
        D_m = int(len(filter_freq)) # Number of kept frequencies
        cost = CostFunctionVSGS(PSD, filter_mask).fit(GFT)
        algo = rpt.Dynp(custom_cost=cost).fit(GFT)
        for d in range(1, d_max+1):
            bkps = algo.predict(n_bkps=d)
            curr_cost = cost.sum_of_costs(bkps)
            bkps_list.append(bkps)
            parameters_list[count] = [lbd, d, D_m, curr_cost]
            count += 1
    K1, K2, K3 = slopeHeuristic(parameters_list, T)
    # Compute full costs with penalty + constants
    final_costs = parameters_list[:,3]
    final_costs += K1 * parameters_list[:,2]
    final_costs += K2 * parameters_list[:,1]
    final_costs += K3 * parameters_list[:,1] * np.log(T / parameters_list[:,1])

    best_params_idx = np.argmin(final_costs)
    return bkps_list[best_params_idx]

In [ ]:
# Results using True PSD
lambda_set = [0,0.0001,0.0005,0.001,0.005,0.01,0.05,0.1,0.5,1.]
T = signal.shape[1]
d_max = int(T/np.log(T))
bkps = VSGS(G, signal.T, PSD, lambda_set, d_max=d_max)
print(f'{len(bkps)} changes points')
metrics(true_bkps, bkps)
rpt.display(signal[:5].T, true_bkps, bkps)
plt.show()

In [ ]:
# Results using PSD estimation through covariance estimation
lambda_set = [0,0.0001,0.0005,0.001,0.005,0.01,0.05,0.1,0.5,1.]
T = signal.shape[1]
d_max = int(T/np.log(T))
bkps = VSGS(G, signal.T, psd_covariance, lambda_set, d_max=d_max)
print(f'{len(bkps)} changes points')
metrics(true_bkps, bkps)
rpt.display(signal[:5].T, true_bkps, bkps)
plt.show()

In [ ]:
# Results using PSD estimation with perraudin methods
lambda_set = [0,0.0001,0.0005,0.001,0.005,0.01,0.05,0.1,0.5,1.]
T = signal.shape[1]
d_max = int(T/np.log(T))
bkps = VSGS(G, signal.T, psd_perraudin, lambda_set, d_max=d_max)
print(f'{len(bkps)} changes points')
metrics(true_bkps, bkps)
rpt.display(signal[:5].T, true_bkps, bkps)
plt.show()

# Kernel methods

In [ ]:
def Kernel(signal):
    # Slope heuristics to estimate penalty
    T, d = signal.shape
    k_min, k_max = 10, min(int(0.4*T), T//6)
    X = np.arange(k_min,k_max+1)
    y = []
    algo = rpt.Dynp(model="l2").fit(signal)
    cost = rpt.costs.CostL2().fit(signal)
    for k in range(k_min,k_max+1):
        curr_bkps = algo.predict(n_bkps=k)
        y.append(cost.sum_of_costs(curr_bkps))
    slope = LinearRegression().fit(np.expand_dims(X,-1), y).coef_[0]
    pen = - 2 * slope
    
    # Apply kernel change point detection
    algo = rpt.KernelCPD(kernel="linear").fit(signal)
    bkps = algo.predict(pen=pen)
    return bkps
    
# bkps = Kernel(signal.T @ G.U)
bkps = Kernel(signal.T)
rpt.display(signal[:5].T, true_bkps, bkps)
plt.show()

# Benchmark

In [ ]:
def benchmark(method='VSGS', dataset='ER', PSD_type='true', n=100):
    """
    Compute mean performance of given setups over n runs
    
    Parameters
    ----------
    method: 'VSGS', 'LGS', 'Kernel'
    dataset: 'ER', 'BA'
    PSD_type: 'true', 'cov', 'perraudin'
    """
    metric_history = {
        'precision': [],
        'recall': [],
        'hausdorff': [],
        'randindex': []
    }
    for _ in tqdm(range(n)):
        # Generate Graph
        n_nodes = 500
        if dataset == 'BA':
            G, PSD, true_bkps, mus, signal = generate_barabasi_albert(n_nodes)
        elif dataset == 'ER':   
            G, PSD, true_bkps, mus, signal = generate_erdos_renyi(n_nodes)

        # Get PSD
        w = 40
        if PSD_type == 'cov':
            PSD = estimate_PSD_cov(signal[:,:w], G)
        elif PSD_type == 'perraudin':
            PSD = estimate_PSD_perraudin(signal[:,:w], G)

        # Apply algorithm
        T = signal.shape[1]
        d_max = min(int(T/np.log(T)), T//6)
        if method == 'LGS':
            bkps = LGS(G, signal.T, PSD, d_max=d_max, lbd=0.5, verbose=False)
            p, r, hausdorff, randindex = metrics(true_bkps, bkps, verbose=False)
        elif method == 'VSGS':
            lambda_set = [0,0.0001,0.0005,0.001,0.005,0.01,0.05,0.1,0.5,1.]
            bkps = VSGS(G, signal.T, PSD, lambda_set, d_max=d_max, verbose=False)
            p, r, hausdorff, randindex = metrics(true_bkps, bkps, verbose=False)
        elif method == 'Kernel':
            bkps = Kernel(signal.T)
            p, r, hausdorff, randindex = metrics(true_bkps, bkps, verbose=False)
            
        # Record metrics
        metric_history['precision'].append(p)
        metric_history['recall'].append(r)
        metric_history['hausdorff'].append(hausdorff)
        metric_history['randindex'].append(randindex)
    return metric_history

def analyse_benchmark(metric_history):
    for k,v in metric_history.items():
        data = np.array(v)
        print(f'{k:10}: mean {np.mean(v):.2f}, std {np.std(v):.3f}, median {np.median(v):.2f}')

In [ ]:
# Graph methods benchmarks
parameters = {
    'method': ['LGS', 'VSGS'],
    'dataset': ['ER', 'BA'],
    'PSD_type': ['true', 'cov', 'perraudin']
}
for method in parameters['method']:
    for dataset in parameters['dataset']:
        for PSD_type in parameters['PSD_type']:
            print(f'method: {method}, dataset: {dataset}, PSD estimation: {PSD_type}')
            metric_history = benchmark(method=method, dataset=dataset, PSD_type=PSD_type, n=100)
            analyse_benchmark(metric_history)
            print(f'-----------------------')

In [ ]:
# Kernel methods benchmarks
parameters = {
    'method': ['Kernel'],
    'dataset': ['ER', 'BA'],
}

for dataset in parameters['dataset']:
        print(f'method: linear kernel, dataset: {dataset}')
        metric_history = benchmark(method='Kernel', dataset=dataset, n=100)
        analyse_benchmark(metric_history)
        print(f'-----------------------')

# Application to real data

In [ ]:
from loadmydata.load_molene_meteo import load_molene_meteo_dataset
import geopandas
import contextily as cx

CRS = "EPSG:4326"

In [ ]:
# Load data
data_df, stations_df, description = load_molene_meteo_dataset()

In [ ]:
### Preprocessing of temperature dataframe

data_df["temp"] = data_df.t - 273.15  # temperature in Celsius

temperature_df = data_df.pivot(
    index="date", values="temp", columns="station_name"
)

# Drop NaN values
temperature_df = temperature_df.dropna(axis=1)

print(f"The stations with missing values are {', '.join(sorted(list((set(data_df['station_name'])  - set(temperature_df.columns))))).title()}") 

temperature_df.head()

In [ ]:
### Preprocessing of stations dataframe

stations_df_filtered = stations_df[stations_df['Nom'].isin(list(temperature_df.columns))]

# convert pandas df to geopandas df
stations_gdf = geopandas.GeoDataFrame(
    stations_df_filtered,
    geometry=geopandas.points_from_xy(
        stations_df_filtered.Longitude, stations_df_filtered.Latitude
    ),
).set_crs(CRS)

stations_gdf.head()

In [ ]:
from scipy.spatial.distance import pdist, squareform
from math import asin, cos, radians, sin, sqrt
from pygsp import graphs

def get_geodesic_distance(point_1, point_2) -> float:
    """
    Calculate the great circle distance (in km) between two points
    on the earth (specified in decimal degrees)

    https://stackoverflow.com/a/4913653
    """

    lon1, lat1 = point_1
    lon2, lat2 = point_2

    # convert decimal degrees to radians
    lon1, lat1, lon2, lat2 = map(radians, [lon1, lat1, lon2, lat2])

    # haversine formula
    dlon = lon2 - lon1
    dlat = lat2 - lat1
    a = sin(dlat / 2) ** 2 + cos(lat1) * cos(lat2) * sin(dlon / 2) ** 2
    c = 2 * asin(sqrt(a))
    r = 6371  # Radius of earth in kilometers. Use 3956 for miles
    return c * r

In [ ]:
# Compute geodedic distances between all stations 
stations_np = stations_df_filtered[["Longitude", "Latitude"]].to_numpy()
dist_mat_condensed = pdist(stations_np, metric=get_geodesic_distance) # Distances computed in km

# Compute similarity using gaussian smoothing and median heuristic for bandwith parameter
bandwidth = np.median(dist_mat_condensed) # median heuristic
exp_similarity = np.exp( -(dist_mat_condensed ** 2) / np.square(bandwidth))

# Performs dichotomy to find the minimum threshold that keeps the graph connected with an average degree of at least 3
low = 0
high = 1
threshold = 1
eps = 1e-5
while np.abs(high - low) > eps:
    threshold = (high + low) / 2
    adjacency_matrix_gaussian = squareform(np.where(exp_similarity > threshold, exp_similarity, 0.0))
    G = graphs.Graph(adjacency_matrix_gaussian)
    is_connected = G.is_connected(recompute=True)
    if is_connected:
        low = threshold
    else:
        high = threshold

adjacency_matrix_gaussian = squareform(np.where(exp_similarity > low, exp_similarity, 0.0))
G = graphs.Graph(adjacency_matrix_gaussian)
is_connected = G.is_connected(recompute=True)

print(f"The minimum threshold so that the graph is connected is {low:.4f}.")
print(f"The average degree of the graph is {np.mean(G.d):.2f}.")
print(f"The average weighted degree of the graph is {np.mean(np.sum(adjacency_matrix_gaussian, axis=0)):.2f}")

In [ ]:
# Plot the graph
ax = stations_gdf.geometry.plot(figsize=(10, 10))
cx.add_basemap(ax, crs=stations_gdf.crs.to_string(), zoom=8)
ax.set_axis_off()
G.set_coordinates(stations_np)
G.plot(ax=ax)

In [ ]:
temperature_df_np = np.array(temperature_df)
rpt.display(temperature_df_np[:,:5], [])
plt.show()

In [ ]:
# Compute PSD
w = 80
psd_covariance_temp = estimate_PSD_cov(temperature_df_np[:w].T, G)
psd_perraudin_temp = estimate_PSD_perraudin(temperature_df_np[:w].T, G)

fig = plt.figure(figsize=(10, 5)) 
plt.plot(G.e, psd_covariance_temp, label="PSD covariance")
plt.plot(G.e, psd_perraudin_temp, label="PSD peraudin")
plt.xlabel("lambda")
plt.ylabel("PSD")
plt.legend()
plt.show()

## LGS

In [ ]:
# Results Covariance PSD
T = temperature_df_np.shape[0]
d_max = int(T/np.log(T))
bkps = LGS(G, temperature_df_np, psd_covariance_temp, d_max=d_max, lbd=10.)
print(f'{len(bkps)} changes points')

rpt.display(temperature_df_np[:,:5], bkps)
plt.xticks(np.arange(0, 31) * 24, np.arange(1, 32))
plt.xlabel('Date (-- January 2014)')
plt.ylabel('Temperature (°C)')
plt.show()

In [ ]:
# Results Covariance PSD
T = temperature_df_np.shape[0]
d_max = int(T/np.log(T))
bkps = LGS(G, temperature_df_np, psd_perraudin_temp, d_max=d_max, lbd=10.)
print(f'{len(bkps)} changes points')

rpt.display(temperature_df_np[:,:5], bkps)
plt.xticks(np.arange(0, 31) * 24, np.arange(1, 32))
plt.xlabel('Date (-- January 2014)')
plt.ylabel('Temperature (°C)')
plt.show()

## VSGS

In [ ]:
# Results using Covariance PSD
lambda_set = [0,0.0001,0.0005,0.001,0.005,0.01,0.05,0.1,0.5,1.]
T = temperature_df_np.shape[0]
d_max = int(T/np.log(T))
bkps = VSGS(G, temperature_df_np, psd_covariance_temp, lambda_set, d_max=d_max)
print(f'{len(bkps)} changes points')
rpt.display(temperature_df_np[:,:5], bkps)
plt.xticks(np.arange(0, 31) * 24, np.arange(1, 32))
plt.xlabel('Date (-- January 2014)')
plt.ylabel('Temperature (°C)')
plt.show()

In [ ]:
# Results using Peraudin PSD
lambda_set = [0,0.0001,0.0005,0.001,0.005,0.01,0.05,0.1,0.5,1.]
T = temperature_df_np.shape[0]
d_max = int(T/np.log(T))
bkps = VSGS(G, temperature_df_np, psd_perraudin_temp, lambda_set, d_max=d_max)
print(f'{len(bkps)} changes points')
rpt.display(temperature_df_np[:,:5], bkps)
plt.xticks(np.arange(0, 31) * 24, np.arange(1, 32))
plt.xlabel('Date (-- January 2014)')
plt.ylabel('Temperature (°C)')
plt.show()